### 机器学习实战之信用卡盗刷检测###

### 前言###
本位将采样机器学习方法分析一个实际问题：信用卡盗刷，基于历史数据训练模型，得到一个能够对信用卡盗刷进行检测的模型，本文采样noteBook的形式记录整个分析过程，欢迎指正和相互学习。
### 1数据介绍###
数据集来自Kaggle官网，数据集包含由欧洲持卡人于2013年9月使用信用卡进行交的数据。此数据集显示两天内发生的交易，其中284,807笔交易中有492笔被盗刷。数据集非常不平衡，被盗刷占所有交易的0.17％。

数据集一共31个特征，class为标签，1代表被盗刷，0代表正常交易。Time为时间特征，表示的是与第一笔交易的间隔时间，单位：s，Amount为交易的花费，除上述特征外，另有V1~V28个经过数据处理的特征，出于信息保密，这些特征的具体含义无从知晓，但并不影响我们进行分析。下面就让我们来看看具体的数据。

